In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import manhattan_distances
from scipy.spatial.distance import cityblock
from scipy.spatial.distance import pdist
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
def load_glove_embeddings(glove_file_path='glove.6B.300d.txt'):
    embeddings = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        # Find the index of the first numerical token
        embedding_start_index = None
        for i, token in enumerate(first_line.split()):
            try:
                float(token)  # Attempt to convert the token to a float
                embedding_start_index = i
                break
            except ValueError:
                continue
        if embedding_start_index is None:
            raise ValueError("Could not determine the start of the embedding vector in the first line.")
        
        embedding_dim = len(first_line.split()) - embedding_start_index  # Determine the embedding dimension
        print(f"Embedding dimension: {embedding_dim}")
        
        # Reset the file pointer to the beginning of the file
        f.seek(0)
        for line in f:
            values = line.strip().split()
            word = ' '.join(values[:-embedding_dim])  # Reconstruct the word
            vector = list(map(float, values[-embedding_dim:]))  # Extract the embedding vector
            embeddings[word] = vector

    return embeddings

# Load the embeddings
embeddings = load_glove_embeddings(glove_file_path='embeddings_countries.txt')

Embedding dimension: 300


In [3]:
# This code is not strictly necessary, but it only serves to see if the embeddings make sense

def get_top_similarities(target_word, embeddings, top_n=20):
    target_embedding = embeddings.get(target_word)
    if target_embedding is None:
        return None

    similarities = {}
    for word, word_embedding in embeddings.items():
        similarities[word] = -cityblock(target_embedding, word_embedding)

    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return sorted_similarities[:top_n]



# example usage
top_20_similarities = get_top_similarities('åland islands', embeddings)
print(top_20_similarities)

top_20_similarities = get_top_similarities('bangladesh', embeddings)
print(top_20_similarities)

[('åland islands', -0.0), ('faroe islands', -49.706144089999995), ('cayman islands', -56.54005485), ('solomon islands', -59.75879071), ('marshall islands', -61.417534605), ('northern mariana islands', -63.68305945166667), ('cocos (keeling) islands', -66.449814969), ('cook islands', -66.69857771), ('turks and caicos islands', -67.32165119500002), ('falkland islands (malvinas)', -67.854533625), ('virgin islands (british)', -68.38514268166668), ('heard island and mcdonald islands', -70.663665195), ('virgin islands (u.s.)', -71.39904455266667), ('united states minor outlying islands', -73.85662849299999), ('south georgia and the south sandwich islands', -81.29605458071428), ('bouvet island', -81.565424485), ('norfolk island', -82.820635275), ('antigua and barbuda', -83.66415023833332), ('christmas island', -83.969834265), ('british indian ocean territory', -84.5136254225)]
[('bangladesh', -0.0), ('pakistan', -86.06863594000001), ('sri lanka', -86.60521655), ('india', -89.79430434), ('nepal

In [4]:
def bisecting_kmeans(embeddings, num_clusters):
    # Initialize with all data in one cluster
    clusters = {0: list(embeddings.keys())}
    current_num_clusters = 1

    while current_num_clusters < num_clusters:
        # Find the largest cluster to split
        largest_cluster_label = max(clusters, key=lambda label: len(clusters[label]))
        largest_cluster = clusters.pop(largest_cluster_label)

        # Extract embeddings for the largest cluster
        largest_cluster_embeddings = [embeddings[word] for word in largest_cluster]

        # Perform K-means with K=2 on the largest cluster
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        kmeans.fit(largest_cluster_embeddings)

        # Split the cluster into two new clusters
        new_cluster_1 = [largest_cluster[i] for i, label in enumerate(kmeans.labels_) if label == 0]
        new_cluster_2 = [largest_cluster[i] for i, label in enumerate(kmeans.labels_) if label == 1]

        # Add the new clusters to the clusters dictionary
        clusters[current_num_clusters] = new_cluster_1
        clusters[current_num_clusters + 1] = new_cluster_2

        # Update the number of clusters
        current_num_clusters += 2

    # Reassign cluster labels to be continuous from 0 to num_clusters - 1
    new_clusters = {i: clusters[label] for i, label in enumerate(sorted(clusters.keys()))}

    return new_clusters

# Example usage

number_of_words = len(embeddings.keys())
avg_words_per_cluster = 10
num_clusters = number_of_words / avg_words_per_cluster
clusters = bisecting_kmeans(embeddings, num_clusters)

# Print clusters
for label, words in clusters.items():
    print(f"Cluster {label}: {', '.join(words)}")

Cluster 0: argentina, brazil, chile, colombia, costa rica, cuba, dominican republic, ecuador, el salvador, guatemala, haiti, honduras, mexico, nicaragua, paraguay, peru, uruguay
Cluster 1: australia, bangladesh, bhutan, cambodia, china, hong kong, india, indonesia, japan, malaysia, myanmar, nepal, pakistan, philippines, singapore, sri lanka, thailand, vietnam
Cluster 2: bolivia (plurinational state of), bosnia and herzegovina, canada, central african republic, congo (republic of the), congo (democratic republic of the), french southern territories, georgia, vatican city state, korea (democratic people's republic of), korea (republic of), lao people's democratic republic, macedonia (the former yugoslav republic of), moldova (republic of), palestine, state of, russian federation, south africa, south sudan, taiwan, province of china, tanzania, united republic of, united kingdom of great britain and northern ireland, united states of america, venezuela (bolivarian republic of), western sah

In [5]:
# Given a word, find the cluster to which it belongs
def find_word_cluster(word, clusters):
    for label, words in clusters.items():
        if word in words:
            return label
    return None

# Example usage
target_word = 'france'
target_cluster = find_word_cluster(target_word, clusters)
if target_cluster is not None:
    print(f"Word '{target_word}' belongs to cluster {target_cluster}.")
else:
    print(f"Word '{target_word}' does not belong to any cluster.")

Word 'france' belongs to cluster 9.


In [6]:
def compute_largest_intercluster_distance(clusters, embeddings):
    K = 1 # amplification factor, we didn't need it bigger than 1

    # Calculate centroids of each cluster
    centroids = {}
    for label, words in clusters.items():
        cluster_vectors = [embeddings[word] for word in words]
        centroids[label] = K * np.mean(cluster_vectors, axis=0)

    # Compute pairwise distances between centroids
    centroid_labels = list(centroids.keys())
    centroid_vectors = [centroids[label] for label in centroid_labels]
    distances = manhattan_distances(centroid_vectors, centroid_vectors)

    # Find the maximum distance
    sensitivity_inter = np.max(distances)
    
    return distances, sensitivity_inter




# Metric DP, sensitivity = 1
def exponential_mechanism_for_cluster(clusters, distances, selected_label, epsilon=1):
    # Calculate the utilities for each cluster based on the negative distance to the selected cluster
    utilities = np.array([-distances[selected_label, label] for label in clusters.keys()])

    # sensitivity = np.max(distances)
    sensitivity = 1

    # Compute the probabilities for each cluster using the exponential mechanism
    probabilities = np.exp(utilities * epsilon / (2 * sensitivity))
    probabilities /= np.sum(probabilities)

    # Select a cluster probabilistically
    selected_cluster = np.random.choice(list(clusters.keys()), p=probabilities)

    return selected_cluster


distances_inter, sensitivity_inter = compute_largest_intercluster_distance(clusters, embeddings)
print(f"Max intercluster distance: {sensitivity_inter}")

selected_label = target_cluster  # The label of the cluster for which we want to select a similar cluster
selected_cluster = exponential_mechanism_for_cluster(clusters, distances_inter, selected_label, epsilon=1)
print(f"Selected Cluster {selected_cluster}: {', '.join(clusters[selected_cluster])}")

Max intercluster distance: 88.66460151975168
Selected Cluster 9: austria, belgium, denmark, finland, france, germany, iceland, ireland, italy, luxembourg, netherlands, norway, portugal, spain, sweden, switzerland


In [7]:
def compute_largest_intracluster_distance(clusters, embeddings):
    sensitivity_intra = {}
    for label, words in clusters.items():
        # Get the embeddings for the words in the cluster
        cluster_embeddings = [embeddings[word] for word in words]
        
        # Compute the pairwise distances within the cluster
        if len(cluster_embeddings) > 1:
            distances = pdist(cluster_embeddings, metric='cityblock')
            # Find the maximum distance
            max_distance = max(distances)
        else:
            # If the cluster has only one element, set the max distance to 0
            max_distance = 0
        
        # Store the result in the dictionary
        sensitivity_intra[label] = max_distance
    
    return sensitivity_intra



def exponential_mechanism_for_word(selected_cluster_embeddings, target_word_embedding, sensitivity_intra, epsilon=1):
    # calculate the utility for each word based on the negative
    utilities = [-cityblock(target_word_embedding, word_embedding) for word_embedding in selected_cluster_embeddings]

    # Compute the probabilities for each word using the exponential mechanism
    probabilities = np.exp(epsilon * np.array(utilities) / (2 * sensitivity_intra))
    probabilities /= np.sum(probabilities)

    # Randomly select a word based on the probabilities
    selected_index = np.random.choice(range(len(selected_cluster_embeddings)), p=probabilities)
    return selected_index





target_word_embedding = embeddings[target_word]
selected_cluster_embeddings = [embeddings[word] for word in clusters[selected_cluster]]
sensitivity_intra = compute_largest_intracluster_distance(clusters, embeddings) # dict

selected_index = exponential_mechanism_for_word(selected_cluster_embeddings, target_word_embedding, sensitivity_intra[selected_cluster], epsilon=1)
selected_word = clusters[selected_cluster][selected_index]

print(f"Selected word: {selected_word}")

Selected word: netherlands


In [10]:
# run everything here

# Load the embeddings
embeddings = load_glove_embeddings(glove_file_path='embeddings_countries.txt')

# create clusters
number_of_words = len(embeddings.keys())
avg_words_per_cluster = 10
num_clusters = number_of_words / avg_words_per_cluster
clusters = bisecting_kmeans(embeddings, num_clusters)


def replace_word(target_word, clusters, embeddings):

    target_cluster = find_word_cluster(target_word, clusters)
    # print(f"Target Cluster {target_cluster}: {', '.join(clusters[target_cluster])}")

    # select cluster
    distances_inter, sensitivity_inter = compute_largest_intercluster_distance(clusters, embeddings)
    selected_label = target_cluster  # The label of the cluster for which we want to select a similar cluster
    selected_cluster = exponential_mechanism_for_cluster(clusters, distances_inter, selected_label, epsilon=1)
    # print(f"Select Cluster {selected_cluster}: {', '.join(clusters[selected_cluster])}")

    # select word
    target_word_embedding = embeddings[target_word]
    selected_cluster_embeddings = [embeddings[word] for word in clusters[selected_cluster]]
    sensitivity_intra = compute_largest_intracluster_distance(clusters, embeddings) # dict

    selected_index = exponential_mechanism_for_word(selected_cluster_embeddings, target_word_embedding, sensitivity_intra[selected_cluster], epsilon=1)
    selected_word = clusters[selected_cluster][selected_index]

    return selected_word

def get_replacement_dict(embeddings, clusters):
    replacement_dict = {}
    for word in embeddings.keys():
        replacement_dict[word] = replace_word(word, clusters, embeddings)
    return replacement_dict

replacement_dict = get_replacement_dict(embeddings, clusters)

print('replacement_dict: ', replacement_dict)


Embedding dimension: 300
replacement_dict:  {'afghanistan': 'iran', 'åland islands': 'falkland islands (malvinas)', 'albania': 'czech republic', 'algeria': 'cameroon', 'american samoa': 'south georgia and the south sandwich islands', 'andorra': 'mayotte', 'angola': 'zimbabwe', 'anguilla': 'guyana', 'antarctica': 'south georgia and the south sandwich islands', 'antigua and barbuda': 'bahamas', 'argentina': 'argentina', 'armenia': 'belarus', 'aruba': 'dominica', 'australia': 'bangladesh', 'austria': 'portugal', 'azerbaijan': 'azerbaijan', 'bahamas': 'montserrat', 'bahrain': 'iraq', 'bangladesh': 'india', 'barbados': 'montserrat', 'belarus': 'lithuania', 'belgium': 'portugal', 'belize': 'montserrat', 'benin': 'ghana', 'bermuda': 'panama', 'bhutan': 'china', 'bolivia (plurinational state of)': 'palestine, state of', 'bonaire, sint eustatius and saba': 'saint lucia', 'bosnia and herzegovina': 'venezuela (bolivarian republic of)', 'botswana': 'uganda', 'bouvet island': 'virgin islands (briti

In [13]:
def translate_text(text, replacement_dict):
    # Sort the keys by length in descending order
    # This is necessary to ensure that longer keys are replaced first
    # E.g., if we have 'mexico city' and 'mexico', we want to replace 'mexico city' first
    sorted_keys = sorted(replacement_dict.keys(), key=len, reverse=True)
    translated_text = text
    for key in sorted_keys:
        # Replace occurrences of the key in the text with its value
        translated_text = translated_text.replace(key, replacement_dict[key])
    return translated_text

text = "I am from bangladesh. I live in the åland islands."
translated_text = translate_text(text, replacement_dict)
print(translated_text)

I am from sri lanka. I live in the falkland islands (malvinas).
